# Loading Custom Modules

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('C:/Users/hkimr/Desktop/RPS Project/RPS-Deep-Learning/modules')

from DataLoaders import *
from Visualizations import *
from TrainTest import *
from MakeDataset import *

# Creating Custom Dataset

In [ ]:
print("Creating Training Set")
createDataSet("train")
print("Creating Validation Set")
createDataSet("val")
print("Creating Testing Set")
createDataSet("test")

# Defining DataLoaders

## Hyperparameters

In [ ]:
batch_size = 150
learning_rate = 0.001
n_epochs = 16
output_neurons = 3
regularization = 0.001
model_name = "Custom_Dataset_With_Val_Loss_And_Augmentation"

In [ ]:
train_transform = defineDataTransform("train")
val_transform = defineDataTransform("val")
test_transform = defineDataTransform("test")

train_loader = createDataLoader("data/rps-train", train_transform, batch_size = batch_size)
val_loader = createDataLoader("data/rps-val", val_transform, batch_size = batch_size)
test_loader = createDataLoader("data/rps-test", test_transform, test_loader = True)

## Example Image From Dataset

In [ ]:
showNumpyAsImg(next(iter(train_loader))[0][0])

# Defining Network Architecture

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 15)
        self.conv2 = nn.Conv2d(6, 16, 15)
        self.pool = nn.MaxPool2d(5, 5)
        
        self.fc1 = nn.Linear(16*8*8, 300)
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, output_neurons)
        
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, 16*8*8)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

# Training The Network

In [ ]:
model = CNN()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = regularization)
trainNetwork(model, train_loader, val_loader, n_epochs, optimizer)

# Saving The Model

In [ ]:
torch.save(model.state_dict(), f'saved_models/{model_name}_{batch_size}BS_{learning_rate}LR_{n_epochs}E')

# Evaluating The Model

In [ ]:
model = CNN()
loadModel(model, f'{model_name}_{batch_size}BS_{learning_rate}LR_{n_epochs}E')
evaluateNetwork(model, test_loader)